# 판다스와 데이터 타입
판다스는 `numpy`를 확장해서 다양한 데이터 타입을 사용할 수 있습니다.

| dtype | 설명 | 
| -- | -- |
| object | 문자열 |
| int64, int32, int16, int8 | 부호 있는 정수 |
| uint64, uint32, uint16, uint8 | 부호 없는 정수 |
| float64, float32, float16 | 부동소수점 |
| bool | 참과 거짓 판별 | 
| datetime64 | 날짜 |
| timedelta | 시간차 |
| category | 범주형 데이터 |

다음 예제 데이터로 실습해 봅시다.

In [ ]:
from pandas import DataFrame
import numpy as np

df = DataFrame({
    "x": ["apple", "banana"] * 100 ,
    "y": ["20210101", "20210102"] * 100,
    "z": np.arange(200)
})
df

,x,y,z
0,apple,20210101,0
1,banana,20210102,1
2,apple,20210101,2
3,banana,20210102,3
4,apple,20210101,4
...,...,...,...
195,banana,20210102,195
196,apple,20210101,196
197,banana,20210102,197
198,apple,20210101,198


데이터 프레임의 `info` 메서드는 데이터 타입, 메로리 사용 현황 등의 정보를 출력합니다.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x       200 non-null    object
 1   y       200 non-null    object
 2   z       200 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


기본적으로 메모리 사용량은 대략적인 추정치가 반환됩니다. 정확한 메모리 사용량을 확인하려면 `memory_usage="deep"` 옵션을 추가해야 합니다.

In [ ]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x       200 non-null    object
 1   y       200 non-null    object
 2   z       200 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 26.6 KB


데이터프레임에 `memory_usage` 메서드가 존재합니다. 이는 컬럼 단위로 대략적인 메모리 사용량(추정치)를 보여줍니다. 단위는 `byte` 입니다. 

In [ ]:
df.memory_usage()

Index     128
x        1600
y        1600
z        1600
dtype: int64

`deep` 옵션을 추가하면 정확한 사용량을 계산합니다. 

전체 합을 계산하면 `info` 메서드에서 본 결과를 확인할 수 있습니다. 

## category
`object`로 표현된 `apple`은 5 byte(40 bits) 이상의 메모리가 필요하며, 이러한 apple이 사용된 횟수에 비례해서 메모리 크기가 증가합니다. `category` 타입은 범주형 데이터를 효과적으로 표현합니다. 전체 컬럼에서 사용된 데이터를 확인하고 각각의 데이터를 인코딩해서 사용되는 메모리를 줄일 수 있습니다. `apple`을 0으로, `banana`를 1로 변환해서 저장하면 1 bit로 각 데이터를 구분할 수 있습니다. 


In [ ]:
how = {   
    "x"  : 'category',
    "y" : np.array(100)
}
t = df.astype(how)

형변환한 데이터프레임의 크기를 비교해 봅시다.

In [ ]:
df.astype(how)

,x,y,z
0,apple,20210101,0
1,banana,20210102,1
2,apple,20210101,2
3,banana,20210102,3
4,apple,20210101,4
...,...,...,...
195,banana,20210102,195
196,apple,20210101,196
197,banana,20210102,197
198,apple,20210101,198


## datetime64
날짜 표현에 효과적인 데이터 타입입니다. `datetime64` 형식으로 날짜를 표현하면 `연/월/주/일/시/분/초/분기` 정보를 쉽게 계산할 수 있습니다. 

### Timestamp
판다스의 `to_datetime` 메서드는 문자열로 표현된 날짜를 `Timestamp` 객체로 형변환합니다.

In [ ]:
import pandas as pd

data = pd.to_datetime("20210101")
print(type(data))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


`Timestamp` 객체에는 `year`, `month`, `day`, `minute`, `second`, `quarter` 인스턴스 변수가 정의돼 있습니다.

In [ ]:
print(data.year, data.month, data.day, data.minute, data.second, data.quarter)

2021 1 1 0 0 1


어떻게 만들어져 있을까요? 원리를 이해해 봅시다. 

In [ ]:
class MyTimestamp:
    def __init__(self, val):
        self.year = int(val[:4])
        self.month = int(val[4:6])
m = MyTimestamp("20210101")
m.year

2021

영어권, 유럽 등에서는 시간을 표현하는 순서가 반대이거나 (우리 입장에서) 뒤죽박죽일 수 있습니다. `일-월-년` 형태로 문자열 날짜가 표현돼 있다면 어떻게 형변환을 할 수 있을까요?

In [ ]:
data = pd.to_datetime("05-01-2021")
data

Timestamp('2021-05-01 00:00:00')

2021년 1월 5일을 표현하고 싶었는데 우리의 의도를 알 수 없는 판다스는 5월 1일로 해석해 버립니다. `format` 파라미터는 판다스에게 문자열에서 날짜 데이터가 구성된 형태를 전달해 줍니다. 
- https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
data = pd.to_datetime("05-01-2021")
data

Timestamp('2021-05-01 00:00:00')

In [ ]:
data = pd.to_datetime("05-01-2021", format = '%d-%M-%Y')
data

Timestamp('2021-01-05 00:01:00')

In [ ]:
data = pd.to_datetime("05@@@@01####2021", format="%d@@@@%M####%Y")
data

Timestamp('2021-01-05 00:01:00')

### DatetimeIndex

`to_datetime`은 여러 개의 문자열 날짜 타입을 한 번에 `DatetimeIndex`로 형변환할 수 있습니다.

In [ ]:
data = pd.to_datetime(["20210101", "20210102", "20210103"])
print(type(data))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


`DatetimeIndex`은 리스트와 유사한 순서가 있는 자료구조의 하나로 여러 개의 `Timestamp` 객체를 관리합니다. 인덱싱과 슬라이싱을 사용할 수 있습니다.

In [ ]:
data[0]

Timestamp('2021-01-01 00:00:00')

In [ ]:
data

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03'], dtype='datetime64[ns]', freq=None)

`DatetimeIndex` 객체에도 `year`, `month`, `day`, `minute`, `second`, `quarter` 인스턴스 변수가 정의돼 있습니다. `Timestamp`와 달리 반환 결과는 자료구조입니다. 

In [ ]:
data.year

Int64Index([2021, 2021, 2021], dtype='int64')

In [ ]:
data.quarter

Int64Index([1, 1, 1], dtype='int64')

### Series와 DatetimeIndex

`y` 컬럼을 `DatetimeIndex`로 변환해 봅시다.

In [ ]:
df.head()

,x,y,z
0,apple,20210101,0
1,banana,20210102,1
2,apple,20210101,2
3,banana,20210102,3
4,apple,20210101,4


In [ ]:
df['y']

0      20210101
1      20210102
2      20210101
3      20210102
4      20210101
         ...   
195    20210102
196    20210101
197    20210102
198    20210101
199    20210102
Name: y, Length: 200, dtype: object

In [ ]:
data = pd.to_datetime(df.y)
print(type(data))

<class 'pandas.core.series.Series'>


첫 번째 데이터의 타입을 확인해 볼까요?

In [ ]:
print(type(data.iloc[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Series에 저장된 `Timestamp`는 년/월/일/분기 정보에 접근할 때 `dt` 인스턴스 변수(속성)를 사용해야 합니다. 코드를 실행한 결과는 시리즈로 반환됩니다.

In [ ]:
data.dt.year

0      2021
1      2021
2      2021
3      2021
4      2021
       ... 
195    2021
196    2021
197    2021
198    2021
199    2021
Name: y, Length: 200, dtype: int64

In [ ]:
data.dt.quarter

0      1
1      1
2      1
3      1
4      1
      ..
195    1
196    1
197    1
198    1
199    1
Name: y, Length: 200, dtype: int64

DatetimeIndex를 시리즈로 직접 타입을 변경할 수도 있습니다.
- `to_series`

In [ ]:
data = pd.to_datetime(["20210101", "20210102"])

In [ ]:
df['y'] = pd.to_datetime( df['y'] )
df['mon'] = df['y'].dt.month
df['year'] = df['y'].dt.year

how = {
    'z' : np.mean
}
df.groupby(by= ['year', 'mon']).agg(how)

,,z
year,mon,
2021,1,99.5


## Timedelta
시간 차이를 표현하는 데이터 타입입니다. 

In [ ]:
td = pd.Timedelta(days=10)
print(type(td))

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


In [ ]:
td = pd.Timedelta(minutes=10)
print(type(td))

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


`Timestamp`와 함께 사용하면 시간차를 쉽게 계산할 수 있습니다. 

In [ ]:
import pandas as pd

a = pd.to_datetime("2022-05-31")
b = pd.Timedelta(days=1000)
c = a + b
c.year, c.month, c.day

(2025, 2, 24)

In [ ]:
td = pd.Timedelta(minutes=10)
print(type(td))

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


In [ ]:
td = pd.Timedelta(days=10, minutes=10, hours=9)
td

Timedelta('10 days 09:10:00')

시리즈에 연산을 적용하면 브로드캐스팅돼 전체 데이터에 확장 적용되겠죠?

## date_range
판다스에 정의된 함수로 정해진 구간의 `DatetimeIndex`를 생성합니다. 

In [ ]:
a = pd.date_range(start="20210101", end="20211231")
a

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [ ]:
a + pd.Timedelta(days=1000)

DatetimeIndex(['2023-09-28', '2023-09-29', '2023-09-30', '2023-10-01',
               '2023-10-02', '2023-10-03', '2023-10-04', '2023-10-05',
               '2023-10-06', '2023-10-07',
               ...
               '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20',
               '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24',
               '2024-09-25', '2024-09-26'],
              dtype='datetime64[ns]', length=365, freq='D')

`freq` 파라미터로 주기를 설정할 수 있습니다.

In [ ]:
pd.date_range(start="20210101", end="20211231", freq='3d') # 3일단위로 날짜 만들기

DatetimeIndex(['2021-01-01', '2021-01-04', '2021-01-07', '2021-01-10',
               '2021-01-13', '2021-01-16', '2021-01-19', '2021-01-22',
               '2021-01-25', '2021-01-28',
               ...
               '2021-12-03', '2021-12-06', '2021-12-09', '2021-12-12',
               '2021-12-15', '2021-12-18', '2021-12-21', '2021-12-24',
               '2021-12-27', '2021-12-30'],
              dtype='datetime64[ns]', length=122, freq='3D')

## 연습문제

### 연습문제-1
오늘 날짜는 `2022-05-28` 입니다. 두 달뒤인 7월의 마지막 일자를 출력하세요.

In [ ]:
a = pd.to_datetime("2022-05-28")
b = pd.Timedelta(days=64)
c = a + b
c

Timestamp('2022-07-31 00:00:00')

In [ ]:
# 다른방법1) 월별 날짜수가 다르니까 8월에서 하루 빼는게 편하겠지?
# 8월 첫날 구하기
new_d = pd.to_datetime(f'{a.year}{a.month+3:02}01') # :02 넣으면 두자리수로 고정됨.
new_d - pd.Timedelta(days=1)

Timestamp('2022-07-31 00:00:00')

In [ ]:
# 다른방법2) 직접 타임스템프 만들기
newd = pd.Timestamp(year=a.year, month=a.month+3, day=1)
newd - pd.Timedelta(days=1)

Timestamp('2022-07-31 00:00:00')

In [ ]:
# 다른방법3)
a = pd.date_range("2022-05-28","2022-12-31")
a[a.month == 7][-1]

Timestamp('2022-07-31 00:00:00', freq='D')

### 연습문제-2
변수 `d`에는 2021년의 `Timestamp`가 `DatatimeIndex`에 저장돼 있습니다. 


In [ ]:
d = pd.date_range(start="20210101", end="20211231")
d

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

`numpy`의 `random` 모듈의 `randint` 함수는 임의의 숫자를 생성합니다. 다음 코드는 0부터 9까지의 임의의 숫자를 5개 만듭니다.

In [ ]:
np.random.randint(10, size=5)

위 코드를 응용해서 다음의 데이터프레임을 만들어 보세요.
- x 컬럼에 변수 `d`의 DatetimeIndex를 저장합니다.
- y 컬럼에 0부터 10까지의 임의의 숫자를 저장합니다.

`concat`, `to_series` 등 다양한 방법으로 문제를 해결해 보세요

In [ ]:
# 방법1)dataframe
idx = np.random.randint(11, size=len(d))
df1 = DataFrame({'x':d})
df2 = DataFrame({'y':idx})
df = pd.concat([df1,df2], axis=1)
df

,x,y
0,2021-01-01,4
1,2021-01-02,5
2,2021-01-03,10
3,2021-01-04,3
4,2021-01-05,1
...,...,...
360,2021-12-27,7
361,2021-12-28,10
362,2021-12-29,10
363,2021-12-30,5


In [ ]:
# 방법2) Series
from pandas import Series

s0 = Series(d)
s1 = Series(idx)
s1.head(2)
a = pd.concat( [ s0, s1  ] , axis=1 )
a

,0,1
0,2021-01-01,4
1,2021-01-02,5
2,2021-01-03,10
3,2021-01-04,3
4,2021-01-05,1
...,...,...
360,2021-12-27,7
361,2021-12-28,10
362,2021-12-29,10
363,2021-12-30,5


In [ ]:
# 방법3) to_frame(), to_series
dfx = d.to_frame()
dfx['y'] = idx
dfx = dfx.rename(columns={0:'x'}).reset_index(drop=True)
dfx = dfx.set_index('x')
dfx.index.name='date'
dfx

,y
date,
2021-01-01,4
2021-01-02,5
2021-01-03,10
2021-01-04,3
2021-01-05,1
...,...
2021-12-27,7
2021-12-28,10
2021-12-29,10


In [ ]:
# 방법4) 
dfxx = DataFrame({'x':d, 'y':idx})
dfxx

,x,y
0,2021-01-01,4
1,2021-01-02,5
2,2021-01-03,10
3,2021-01-04,3
4,2021-01-05,1
...,...,...
360,2021-12-27,7
361,2021-12-28,10
362,2021-12-29,10
363,2021-12-30,5


### 연습문제-3
위 문제에서 얻은 데이터프레임에서 년/월/일/분기 컬럼으로 날짜를 분리해서 저장하세요.

| x | y | year | month | day | quarter | 
| -- | -- | -- | -- | -- | -- |
| 20210101 | 9 | 2021 | 1 | 1 | 1 |
| 20210102 | 9 | 2021 | 1 | 2 | 1 |
| 20210103 | 9 | 2021 | 1 | 3 | 1 |

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   x       365 non-null    datetime64[ns]
 1   y       365 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.8 KB


In [ ]:
df = df.set_index('x')

In [ ]:
df['month'] = df.index.month
df['day'] = df.index.day
df['quarter'] = df.index.quarter
df = df.reset_index()
df

,x,y,month,day,quarter
0,2021-01-01,4,1,1,1
1,2021-01-02,5,1,2,1
2,2021-01-03,10,1,3,1
3,2021-01-04,3,1,4,1
4,2021-01-05,1,1,5,1
...,...,...,...,...,...
360,2021-12-27,7,12,27,4
361,2021-12-28,10,12,28,4
362,2021-12-29,10,12,29,4
363,2021-12-30,5,12,30,4


In [ ]:
# 다른방법
df1 = DataFrame({'x':d})
df2 = DataFrame({'y':idx})
df_new = pd.concat([df1,df2], axis=1)
data = pd.to_datetime(df_new.x)
df_new['year'] = data.dt.year
df_new['month'] = data.dt.month
df_new['quarter'] = data.dt.quarter
df_new

,x,y,year,month,quarter
0,2021-01-01,4,2021,1,1
1,2021-01-02,5,2021,1,1
2,2021-01-03,10,2021,1,1
3,2021-01-04,3,2021,1,1
4,2021-01-05,1,2021,1,1
...,...,...,...,...,...
360,2021-12-27,7,2021,12,4
361,2021-12-28,10,2021,12,4
362,2021-12-29,10,2021,12,4
363,2021-12-30,5,2021,12,4
